In [1]:
import os
import pandas as pd
import math 
from collections import defaultdict 

DATA_DIR="../data"

In [2]:
# Get the list of files in the directory
files = os.listdir(DATA_DIR)

# Print the list of files
for file in files:
    print(file)

super_families_indices.pickle
CD_models_curated_sorted
SPARCLE_IDS_curated.mtx4ml.tsv.gz
curated_names_unique
CD_models_curated
super_families_curated
SPARCLE_IDS_curated.csv
CD_models_indices.pickle
SPARCLE_IDS_curated_simplified.csv
super_families_curated_sorted


## Import preprocessed data

### Import input

In [6]:
# Access &read the file 
file_path = os.path.join(DATA_DIR, files[2])
df_one_hot = pd.read_csv(file_path, sep='\t', compression='gzip')

#test succ. loading
print(df_one_hot.head())

                                 CurName  SpecificArch_CHL00001  \
0            FtsZ/tubulin family protein                      0   
1  TCP-1/cpn60 chaperonin family protein                      0   
2              envelope glycoprotein 120                      0   
3               ABC transporter permease                      0   
4     homeobox domain-containing protein                      0   

   SpecificArch_CHL00002  SpecificArch_CHL00004  SpecificArch_CHL00005  \
0                      0                      0                      0   
1                      0                      0                      0   
2                      0                      0                      0   
3                      0                      0                      0   
4                      0                      0                      0   

   SpecificArch_CHL00008  SpecificArch_CHL00009  SpecificArch_CHL00011  \
0                      0                      0                      0   
1   

In [22]:
# Access &read the file 
file_path = os.path.join(DATA_DIR, files[1])
df_sf_pickle = pd.read_csv(file_path)

#test succ. loading
print(df_sf_pickle)

         CHL00001
0        CHL00002
1        CHL00004
2        CHL00005
3        CHL00008
4        CHL00009
...           ...
28049  smart01096
28050  smart01097
28051  smart01099
28052  smart01100
28053  smart01101

[28054 rows x 1 columns]


### Import labels

In [3]:
# Access &read the file 
file_path = os.path.join(DATA_DIR, files[8])
df_labels_simplified = pd.read_csv(file_path, on_bad_lines  = "warn")

In [4]:
df_selCol = df_labels_simplified[['CurName_simplified', 'superfamilyarch', 'SpecificArch']]
len(df_selCol)

42766

In [5]:
# Remove rows where either 'SpecificArch' or 'CurName' is empty
df_selCol = df_selCol.dropna(subset=['SpecificArch', 'superfamilyarch', 'CurName_simplified'])
df_selCol = df_selCol[(df_selCol['SpecificArch'] != '') & (df_selCol['superfamilyarch'] != '') & (df_selCol['CurName_simplified'] != '')]
len(df_selCol)

39584

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

In [36]:
vectorizer = CountVectorizer(tokenizer=lambda txt: txt.split())

In [37]:
X_SpecificArch = vectorizer.fit_transform(df_selCol['SpecificArch'])

/Users/franzi/Documents/codeathon_NCBI_1/mlxai-2024-team-gwadz-yang/venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [38]:
df_SpecificArch_encoded = pd.DataFrame(X_SpecificArch.toarray(), columns=vectorizer.get_feature_names_out())

In [40]:
df_SpecificArch_encoded.shape

(39584, 28054)

In [41]:
X_superfamilyarch = vectorizer.fit_transform(df_selCol['superfamilyarch'])

/Users/franzi/Documents/codeathon_NCBI_1/mlxai-2024-team-gwadz-yang/venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [42]:
df_superfamilyarch_encoded = pd.DataFrame(X_superfamilyarch.toarray(), columns=vectorizer.get_feature_names_out())

In [43]:
df_superfamilyarch_encoded.shape

(39584, 13139)

In [44]:
df_final = df_selCol.join(df_SpecificArch_encoded).join(df_superfamilyarch_encoded)

ValueError: columns overlap but no suffix specified: Index(['cog0610', 'cog1033', 'cog1099', 'cog1107', 'cog1149', 'cog1202',
       'cog1233', 'cog1315', 'cog1340', 'cog1350',
       ...
       'tigr02099', 'tigr02677', 'tigr02928', 'tigr03083', 'tigr03084',
       'tigr03085', 'tigr03089', 'tigr03172', 'tigr03549', 'tigr04076'],
      dtype='object', length=2153)

In [ ]:
df_final.shape

In [7]:
"""
The code processes rows in a DataFrame, generating two dictionaries (dict_feat and dict_idx_feat). 
dict_feat contains feature keys derived from specific columns, while dict_idx_feat is a nested dictionary 
associating row indices with feature keys and binary values indicating their presence in each row.
"""
dict_feat = defaultdict() 
dict_idx_feat =  defaultdict(dict) 

for index, row in df.iterrows():
    #if type(row["Shortnames"]) == str:
    #    dict_feat["Shortnames_" + row["Shortnames"]] = ""
    #    dict_idx_feat[index]["Shortnames_" + row["Shortnames"]] = 1
    if type(row["SpecificArch"]) == str:
        id_list = row["SpecificArch"].split(" ")
        for id in id_list: 
            dict_feat["SpecificArch_" + id] = ""
            dict_idx_feat[index]["SpecificArch_" + id] = 1
    if type(row["superfamilyarch"]) == str:
        dict_feat["superfamilyarch_" + row["superfamilyarch"]] = ""
        dict_idx_feat[index]["superfamilyarch_" + row["superfamilyarch"]] = 1

In [10]:
dict_idx_feat

defaultdict(dict,
            {0: {'SpecificArch_COG1021': 1, 'superfamilyarch_AMP-binding': 1},
             1: {'SpecificArch_TIGR02275': 1,
              'superfamilyarch_AMP-binding': 1},
             2: {'SpecificArch_TIGR02275': 1,
              'superfamilyarch_DHB_AMP_lig': 1},
             3: {'SpecificArch_COG1021': 1, 'superfamilyarch_EntE': 1},
             4: {'SpecificArch_PRK10946': 1, 'superfamilyarch_entE': 1},
             5: {'SpecificArch_PRK10946': 1, 'superfamilyarch_PRK07788': 1},
             6: {'SpecificArch_PRK10581': 1,
              'superfamilyarch_Isoprenoid_Biosyn_C1': 1},
             7: {'SpecificArch_cd02980': 1,
              'superfamilyarch_TRX_superfamily': 1},
             8: {'SpecificArch_cl02784': 1,
              'SpecificArch_cd02980': 1,
              'superfamilyarch_TRX_superfamily Chelatase_Class_II': 1},
             9: {'SpecificArch_COG2080': 1, 'superfamilyarch_CoxS': 1},
             11: {'SpecificArch_cl30249': 1,
              'Sp

### Further preprocessing 

Remove rows that do not have any value in either 'SpecificArch', 'superfamilyarch', 'CurName'

In [ ]:
# Remove rows where either 'SpecificArch' or 'CurName' is empty
df = df.dropna(subset=['SpecificArch', 'CurName'])
df = df[(df['SpecificArch'] != '') & (df['CurName'] != '')]

## Split data set into training and testing data

In [34]:
from sklearn.model_selection import train_test_split

In [56]:
#just for test and short running of problem: take first 1000 lines from df
df_final = df.head(5000)

In [57]:
X = df_final.drop(['CurName'], axis=1)  # Features
y = df_final['CurName']  # Target variable

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)

Training set size: (4000, 48296)
Testing set size: (1000, 48296)


In [59]:
X_train.iloc[[29]]
y_train.iloc[[29]]

1598    MgtC/SapB family protein
Name: CurName, dtype: object

## Decision Tree Classifier

In [60]:
from sklearn.tree import DecisionTreeClassifier

In [61]:
# Initialize the Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=42)

In [62]:
# Fit the model on the training data
clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [63]:
# Make predictions and evaluate the model
predictions = clf.predict(X_test)

In [65]:
# Evaluate the model (e.g.: accuracy)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)

print("Accuracy:", accuracy)

Accuracy: 0.152


Further ideas: 
- 'superfamilyarch' seems to be more general, prioritize decision tree formation by first considering 'superfamilyarch' and then 'SpecificArch'?
- Parameter change: 'min_samples_split': determines the minimum number of samples required to split an internal node, increasing this number can help prevent overfitting

## Support Vector Machine (SVM)

In [67]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pandas as pd

In [68]:
# Initialize the SVM classifier
svm_clf = SVC(random_state=42)

In [70]:
# Fit the model on the training data
svm_clf.fit(X_train, y_train)

SVC(random_state=42)

In [71]:
# Make predictions on the test set
predictions = svm_clf.predict(X_test)

In [72]:
# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
accuracy

0.032

## Principal Component Analysis

In [73]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [74]:
# standardize features before applying PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [76]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [78]:
# Initialize PCA - for example, to reduce the dataset to 2 principal components
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [79]:
# The number of components PCA chose to keep
print("Number of components chosen:", pca.n_components_)

Number of components chosen: 3404


Remember values above created have been only done with a subset of 5000 rows so not representative!

In [ ]:
#now re-run PCA or SVM with reduced dimensions!

In [80]:
%%bash 
pip freeze 

anyio==4.3.0
appnope==0.1.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==2.4.1
async-lru==2.0.4
attrs==23.2.0
Babel==2.14.0
beautifulsoup4==4.12.3
bleach==6.1.0
certifi==2024.2.2
cffi==1.16.0
charset-normalizer==3.3.2
comm==0.2.1
debugpy==1.8.1
decorator==5.1.1
defusedxml==0.7.1
exceptiongroup==1.2.0
executing==2.0.1
fastjsonschema==2.19.1
fqdn==1.5.1
h11==0.14.0
httpcore==1.0.4
httpx==0.27.0
idna==3.6
importlib-metadata==7.0.1
ipykernel==6.29.3
ipython==8.18.1
isoduration==20.11.0
jedi==0.19.1
Jinja2==3.1.3
joblib==1.3.2
json5==0.9.17
jsonpointer==2.4
jsonschema==4.21.1
jsonschema-specifications==2023.12.1
jupyter-events==0.9.0
jupyter-lsp==2.2.3
jupyter_client==8.6.0
jupyter_core==5.7.1
jupyter_server==2.12.5
jupyter_server_terminals==0.5.2
jupyterlab==4.1.2
jupyterlab_pygments==0.3.0
jupyterlab_server==2.25.3
MarkupSafe==2.1.5
matplotlib-inline==0.1.6
mistune==3.0.2
nbclient==0.9.0
nbconvert==7.16.1
nbformat==5.9.2
nest-asyncio==1.6.0
notebook==7.1.1
note